## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vizinga,RU,61.0749,50.1031,58.84,50,64,13.06,broken clouds
1,1,Upernavik,GL,72.7868,-56.1549,2.70,76,21,5.39,few clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,54.12,90,40,3.04,scattered clouds
3,3,Cherskiy,RU,68.7500,161.3000,27.55,92,100,17.16,overcast clouds
4,4,Hurricane,US,37.1753,-113.2899,75.67,18,94,3.98,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
ideal_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hurricane,US,37.1753,-113.2899,75.67,18,94,3.98,overcast clouds
8,8,Mahebourg,MU,-20.4081,57.7000,82.26,69,40,11.50,scattered clouds
14,14,Sola,VU,-13.8833,167.5500,81.27,78,75,6.91,broken clouds
20,20,Butaritari,KI,3.0707,172.7902,82.35,78,12,14.34,few clouds
23,23,Carnarvon,AU,-24.8667,113.6333,80.67,54,97,10.36,overcast clouds
25,25,Tomball,US,30.0972,-95.6161,80.60,80,75,5.75,broken clouds
26,26,Garoua,CM,9.3000,13.4000,84.65,57,98,3.18,overcast clouds
27,27,Kieta,PG,-6.2167,155.6333,83.05,76,20,4.52,light rain
28,28,Hithadhoo,MV,-0.6000,73.0833,84.52,76,30,18.16,scattered clouds
31,31,Mogadishu,SO,2.0371,45.3438,87.67,74,75,11.50,broken clouds


In [23]:
# 4a. Determine if there are any empty rows.
ideal_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#N/A

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = ideal_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hurricane,US,75.67,overcast clouds,37.1753,-113.2899,
8,Mahebourg,MU,82.26,scattered clouds,-20.4081,57.7000,
14,Sola,VU,81.27,broken clouds,-13.8833,167.5500,
20,Butaritari,KI,82.35,few clouds,3.0707,172.7902,
23,Carnarvon,AU,80.67,overcast clouds,-24.8667,113.6333,
25,Tomball,US,80.60,broken clouds,30.0972,-95.6161,
26,Garoua,CM,84.65,overcast clouds,9.3000,13.4000,
27,Kieta,PG,83.05,light rain,-6.2167,155.6333,
28,Hithadhoo,MV,84.52,scattered clouds,-0.6000,73.0833,
31,Mogadishu,SO,87.67,broken clouds,2.0371,45.3438,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hurricane,US,75.67,overcast clouds,37.1753,-113.2899,Super 8 by Wyndham Hurricane Zion National Park
8,Mahebourg,MU,82.26,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,Sola,VU,81.27,broken clouds,-13.8833,167.5500,Leumerus Bungalows
20,Butaritari,KI,82.35,few clouds,3.0707,172.7902,Isles Sunset Lodge
23,Carnarvon,AU,80.67,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))